<a href="https://colab.research.google.com/github/u21598012/COS-760-EA/blob/BERT-fine-tuned/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
import optuna
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM,
    AutoModel,
    AutoConfig,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    AutoModelForSequenceClassification, get_scheduler
)
from peft import LoraConfig, get_peft_model
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('processed_data_it3.csv')
df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise,emotions,tokens,stemmed_tokens
0,hau_train_track_a_00001,kotu ta yi hukunci kan shari'ar zaben dan maja...,0,0,0,0,0,1,"('surprise',)","['kotu', 'hukunci', 'kan', ""shari'ar"", 'zaben'...","['kotu', 'hukunci', 'kan', ""shari'ar"", 'zaben'..."
1,hau_train_track_a_00002,"toh fah inji 'yan magana suka ce """"""""ana wata ...",0,0,0,0,0,1,"('surprise',)","['toh', 'fah', 'inji', ""'yan"", 'magana', '``',...","['toh', 'fah', 'inji', ""'yan"", 'magana', ""''"",..."
2,hau_train_track_a_00003,bincike ya nuna yan najeriya sun fi damuwa da ...,0,0,1,0,1,0,"('fear', 'sadness')","['bincike', 'nuna', 'yan', 'najeriya', 'fi', '...","['bincik', 'nuna', 'yan', 'najeriya', 'fi', 'd..."
3,hau_train_track_a_00004,kwamishina ya musanta rahoton masari ya cire k...,0,0,0,0,0,0,(),"['kwamishina', 'musanta', 'rahoton', 'masari',...","['kwamishina', 'musanta', 'rahoton', 'masari',..."
4,hau_train_track_a_00005,innalillahi wa inna ilaihir raji'un: allah ya ...,0,0,0,0,1,0,"('sadness',)","['innalillahi', 'wa', 'inna', 'ilaihir', ""raji...","['innalillahi', 'wa', 'inna', 'ilaihir', ""raji..."


In [4]:
text = df['stemmed_tokens']
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
labels = df[emotion_labels].values.tolist()
df[emotion_labels] = df[emotion_labels].astype(int)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Davlan/bert-base-multilingual-cased-finetuned-hausa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [ ]:
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df['text'].tolist(), df[emotion_labels].values, test_size=0.3, random_state=892
# )

# tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-hausa")


In [6]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.texts)


In [ ]:
# train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
# val_dataset = EmotionDataset(val_texts, val_labels, tokenizer)

In [7]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
        "Davlan/bert-base-multilingual-cased-finetuned-hausa",
        num_labels=len(emotion_labels),
        problem_type="multi_label_classification"
    )
# model = AutoModelForSequenceClassification.from_pretrained(
#     "Davlan/bert-base-multilingual-cased-finetuned-hausa",
#     num_labels=len(emotion_labels),
#     problem_type="multi_label_classification"
# )

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=4,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     save_strategy="no",
#     logging_dir='./logs',
#     logging_steps=10,
#     load_best_model_at_end=True,
#     optim="adamw_torch",
#     seed=3,
#     learning_rate=1.6788168340923137e-05)

# training_args = TrainingArguments(
#     output_dir="./optuna_results", # Use a different output directory for tuning
#     per_device_train_batch_size=4, # You might tune this
#     per_device_eval_batch_size=16,  # You might tune this
#     save_strategy="no",
#     logging_dir='./optuna_logs',    # Use a different logging directory
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1", # Specify the metric to optimize
#     greater_is_better=True, # Specify if a higher metric is better
#     learning_rate=1.6788168340923137e-05,
#     num_train_epochs=4,
#     seed=3
# )

# training_args = TrainingArguments(
#     output_dir="./optuna_results", # Use a different output directory for tuning
#     per_device_train_batch_size=16, # You might tune this
#     per_device_eval_batch_size=16,  # You might tune this
#     save_strategy="no",
#     logging_dir='./optuna_logs',    # Use a different logging directory
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1", # Specify the metric to optimize
#     greater_is_better=True # Specify if a higher metric is better
# )



In [8]:
def compute_metrics(pred):
    logits, labels = pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int()
    labels = torch.tensor(labels).int()
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_micro': f1_score(labels, preds, average='micro'),
        'f1_samples': f1_score(labels, preds, average='samples'),
        'accuracy': (preds == labels).float().mean().item(),
    }

def custom_loss(outputs, labels):
    return BCEWithLogitsLoss()(outputs.logits, labels)

In [14]:
def run_k_fold_cv(df, k=5):
    all_texts = df['text'].tolist()
    all_labels = df[emotion_labels].values

    # Initialize K-Fold
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)

    # Store results for each fold
    fold_results = []
    best_f1 = 0
    best_model = None

    # Iterate through folds
    for fold, (train_idx, val_idx) in enumerate(kfold.split(all_texts)):
        print(f"\n--- Fold {fold+1}/{k} ---")

        # Split data for this fold
        fold_train_texts = [all_texts[i] for i in train_idx]
        fold_train_labels = all_labels[train_idx]
        fold_val_texts = [all_texts[i] for i in val_idx]
        fold_val_labels = all_labels[val_idx]

        # Create datasets
        train_dataset = EmotionDataset(fold_train_texts, fold_train_labels, tokenizer)
        val_dataset = EmotionDataset(fold_val_texts, fold_val_labels, tokenizer)

        training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        save_strategy="no",
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        optim="adamw_torch",
        seed=3,
        learning_rate=1.6788168340923137e-05)

        fold_model = model_init

        trainer = Trainer(
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            model_init=fold_model )

        # Train model
        trainer.train()

        # Evaluate
        metrics = trainer.evaluate()
        fold_results.append(metrics)
        print(f"Fold {fold+1} results:", metrics)

        # Save best model across folds
        if metrics['eval_f1_macro'] > best_f1:
            best_f1 = metrics['eval_f1_macro']
            best_model = fold_model
            trainer.save_model(f"./best_model_fold_{fold+1}")

    # Average results across folds
    avg_results = {}
    for metric in fold_results[0].keys():
        avg_results[metric] = sum(fold[metric] for fold in fold_results) / len(fold_results)
        std_results = np.std([fold[metric] for fold in fold_results])
        print(f"Average {metric}: {avg_results[metric]:.4f} ± {std_results:.4f}")

    # Save the best model and tokenizer
    tokenizer.save_pretrained("./best_model")

    return avg_results, fold_results

# Execute K-Fold Cross Validation
k = 5  # Number of folds
avg_metrics, fold_metrics = run_k_fold_cv(df, k)

# Additional analysis of fold results
for metric_name in ['eval_f1_macro', 'eval_accuracy']:
    metric_values = [metrics[metric_name] for metrics in fold_metrics]
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, k+1), metric_values)
    plt.xlabel("Fold")
    plt.ylabel(metric_name)
    plt.title(f"{metric_name} across {k} folds")
    plt.savefig(f"{metric_name}_folds.png")
    plt.close()


--- Fold 1/5 ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.613800
20,0.497300
30,0.483900
40,0.520400
50,0.441800
60,0.460000
70,0.460200
80,0.467800
90,0.463900
100,0.511800


Fold 1 results: {'eval_loss': 0.310354083776474, 'eval_f1_macro': 0.64157506839744, 'eval_f1_micro': 0.6401869158878505, 'eval_f1_samples': 0.5159285159285159, 'eval_accuracy': 0.8803418874740601, 'eval_runtime': 3.3573, 'eval_samples_per_second': 127.783, 'eval_steps_per_second': 32.169, 'epoch': 4.0}

--- Fold 2/5 ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss
10,0.603000
20,0.497800
30,0.493200
40,0.477900
50,0.507400
60,0.507000
70,0.451700
80,0.427800
90,0.438700
100,0.451100


Fold 4 results: {'eval_loss': 0.3284471035003662, 'eval_f1_macro': 0.6161399636837004, 'eval_f1_micro': 0.6272321428571429, 'eval_f1_samples': 0.508003108003108, 'eval_accuracy': 0.8702408671379089, 'eval_runtime': 3.3492, 'eval_samples_per_second': 128.092, 'eval_steps_per_second': 32.247, 'epoch': 4.0}

--- Fold 5/5 ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.605000
20,0.534900
30,0.489000
40,0.469300
50,0.467600
60,0.472100
70,0.499800
80,0.429300
90,0.467000
100,0.470200


Fold 5 results: {'eval_loss': 0.30700811743736267, 'eval_f1_macro': 0.6515752618600302, 'eval_f1_micro': 0.655367231638418, 'eval_f1_samples': 0.5496503496503496, 'eval_accuracy': 0.881507396697998, 'eval_runtime': 3.4451, 'eval_samples_per_second': 124.525, 'eval_steps_per_second': 31.349, 'epoch': 4.0}
Average eval_loss: 0.3167 ± 0.0112
Average eval_f1_macro: 0.6315 ± 0.0177
Average eval_f1_micro: 0.6361 ± 0.0136
Average eval_f1_samples: 0.5204 ± 0.0151
Average eval_accuracy: 0.8758 ± 0.0058
Average eval_runtime: 3.3483 ± 0.0550
Average eval_samples_per_second: 128.1596 ± 2.0824
Average eval_steps_per_second: 32.2640 ± 0.5243
Average epoch: 4.0000 ± 0.0000


In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init )

# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="optuna",
#     n_trials=20
# )

# print("Best trial:")
# print(best_trial)

trainer.train()

metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

# Save the trained model and tokenizer
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

In [ ]:
# print("Number of records : ", len(df))

# count_anger = df['anger'].value_counts().get(1, 0)
# print("Occurrences of 'anger':", count_anger)

# count_disgust = df['disgust'].value_counts().get(1, 0)
# print("Occurrences of 'disgust':", count_disgust)

# count_fear = df['fear'].value_counts().get(1, 0)
# print("Occurrences of 'fear':", count_fear)

# count_joy = df['joy'].value_counts().get(1, 0)
# print("Occurrences of 'joy':", count_joy)

# count_sadness = df['sadness'].value_counts().get(1, 0)
# print("Occurrences of 'sadness':", count_sadness)

# count_surprise = df['surprise'].value_counts().get(1, 0)
# print("Occurrences of 'surprise':", count_surprise)


In [ ]:
# !pip freeze > requirements.txt